# Model Training

To run this NB, install the DataBook class wheel 
```
pip install <wheel-file-name.whl>
```

In [4]:
from databook.data_book import DataBook
import pickle
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.linear_model import LogisticRegression
print(skl.__version__)

1.0.2


In [5]:
raw_data_path = '../Data-v1/Standard_Databook_ 06 07 2022.csv.json'
formula_ranges = '../Data-v1/areas.txt'

## Loading an Excel file as a DataBook object

In [6]:
dBook = DataBook()
dBook.load_file(raw_data_path)
dBook.get_data(all_columns=True)

,workbookName,sheetName,numRow,numCol,cellAddress,cellValue,cellFormula,cellType
key,,,,,,,,
Cover!A1,Standard_Databook_ 06 07 2022.xlsx,Cover,1,1,A1,None,None,General
Cover!B1,Standard_Databook_ 06 07 2022.xlsx,Cover,1,2,B1,None,None,Time
Cover!C1,Standard_Databook_ 06 07 2022.xlsx,Cover,1,3,C1,None,None,Time
Cover!D1,Standard_Databook_ 06 07 2022.xlsx,Cover,1,4,D1,None,None,Time
Cover!E1,Standard_Databook_ 06 07 2022.xlsx,Cover,1,5,E1,None,None,Time
...,...,...,...,...,...,...,...,...
Sheet12S!L55,Standard_Databook_ 06 07 2022.xlsx,Sheet12S,55,12,L55,None,None,Currency
Sheet12S!M55,Standard_Databook_ 06 07 2022.xlsx,Sheet12S,55,13,M55,None,None,Currency
Sheet12S!N55,Standard_Databook_ 06 07 2022.xlsx,Sheet12S,55,14,N55,None,None,Currency


## Add features

We compute features for each cell, that is, for each cell we process its context.
Here we are moving to a dataframe where each record is a representation of an individual Excel cell

In [7]:
dBook.pre_process_data(for_training=True)
dBook.get_data()

,sheetName,cellAddress,Label,up1_isBlank,up1_isFormula,up1_isSameType,up1_isWeaklyFormulaConsistent,up2_isWeaklyFormulaConsistent,dw1_isBlank,dw1_isFormula,dw1_isSameType,dw1_isWeaklyFormulaConsistent,dw2_isWeaklyFormulaConsistent,nb1_isWeaklyFormulaConsistent,dw1_isSum
key,,,,,,,,,,,,,,,
Cover!AZ1,Cover,AZ1,False,True,False,False,True,True,False,False,False,False,True,True,False
Lead PL!O1,Lead PL,O1,False,True,False,False,True,True,True,False,False,True,True,True,False
Lead PL!C7,Lead PL,C7,False,False,False,False,False,True,False,True,True,True,True,False,False
Lead PL!D7,Lead PL,D7,False,False,False,False,False,True,False,True,True,True,True,False,False
Lead PL!E7,Lead PL,E7,False,False,False,False,False,True,False,True,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ScratchPad_TB!H152,ScratchPad_TB,H152,False,True,False,False,False,True,True,False,False,False,True,True,False
Sheet8S!A28,Sheet8S,A28,False,False,False,False,False,True,True,False,False,False,True,True,False
Sheet4S!A28,Sheet4S,A28,False,False,False,False,False,True,True,False,False,False,True,True,False


## Add negative cases

Negative cases are cells with formula marked as inconsistent and that are actual errors. This is required for training, cause we are missing supervision. Note this is adding the *Label* column to the dataframe (*False* means a positive case, i.e., a cell with an inconsistent formula that is not an error; *True* means a negative case, i.e., a cell with an inconsistent formula that **is an error**).

To do this we have a file where we have manually enlisted *some* ranges holding formula, that is used to generate the negative cases. Note that we didn't provide all the ranges!

In [8]:
dBook.add_positive_cases(formula_ranges)
df=dBook.get_data(all_columns=False).copy(deep=True)
df.reset_index(inplace=True)
df

,key,sheetName,cellAddress,Label,up1_isBlank,up1_isFormula,up1_isSameType,up1_isWeaklyFormulaConsistent,up2_isWeaklyFormulaConsistent,dw1_isBlank,dw1_isFormula,dw1_isSameType,dw1_isWeaklyFormulaConsistent,dw2_isWeaklyFormulaConsistent,nb1_isWeaklyFormulaConsistent,dw1_isSum
0,Cover!AZ1,Cover,AZ1,False,True,False,False,True,True,False,False,False,False,True,True,False
1,Lead PL!O1,Lead PL,O1,False,True,False,False,True,True,True,False,False,True,True,True,False
2,Lead PL!C7,Lead PL,C7,False,False,False,False,False,True,False,True,True,True,True,False,False
3,Lead PL!D7,Lead PL,D7,False,False,False,False,False,True,False,True,True,True,True,False,False
4,Lead PL!E7,Lead PL,E7,False,False,False,False,False,True,False,True,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331,R1!K14,R1,K14,True,False,True,True,False,True,False,True,True,False,True,True,False
3332,R1!K15,R1,K15,True,False,True,True,False,True,False,True,True,False,True,True,False
3333,R1!K16,R1,K16,True,False,True,True,False,True,False,True,True,False,True,True,False
3334,R1!K17,R1,K17,True,False,True,True,False,True,False,True,True,False,True,True,False


In [9]:
df[df.key=="R1!K18"]

,key,sheetName,cellAddress,Label,up1_isBlank,up1_isFormula,up1_isSameType,up1_isWeaklyFormulaConsistent,up2_isWeaklyFormulaConsistent,dw1_isBlank,dw1_isFormula,dw1_isSameType,dw1_isWeaklyFormulaConsistent,dw2_isWeaklyFormulaConsistent,nb1_isWeaklyFormulaConsistent,dw1_isSum
379,R1!K18,R1,K18,False,False,True,True,True,True,False,True,True,False,True,False,False
3335,R1!K18,R1,K18,True,False,True,True,False,True,False,True,True,False,True,False,False


In [10]:
len(df)

3336

In [11]:
df.columns

Index(['key', 'sheetName', 'cellAddress', 'Label', 'up1_isBlank',
       'up1_isFormula', 'up1_isSameType', 'up1_isWeaklyFormulaConsistent',
       'up2_isWeaklyFormulaConsistent', 'dw1_isBlank', 'dw1_isFormula',
       'dw1_isSameType', 'dw1_isWeaklyFormulaConsistent',
       'dw2_isWeaklyFormulaConsistent', 'nb1_isWeaklyFormulaConsistent',
       'dw1_isSum'],
      dtype='object')

## Split traint and test data

We extract the name of the Excel sheet, and we filter the dataframe over sheets that contains some positive cases, to give it to *train-tests-plit*. The only reason we are doing this is that we didn't process all the available ranges with formulawhen adding negative cases!

In [12]:
sheet_names = df[df.Label==True]['sheetName'].unique()
sheet_names

array(['Lead BS', 'R1'], dtype=object)

In [13]:
red_df = df[df['sheetName'].isin(sheet_names)]

In [14]:
features = [c for c in df.columns if c not in ['Label', 'key', 'cellAddress', 'sheetName']]
X = red_df[features]
Y = red_df['Label']

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=.2)

In [16]:
X_train

,up1_isBlank,up1_isFormula,up1_isSameType,up1_isWeaklyFormulaConsistent,up2_isWeaklyFormulaConsistent,dw1_isBlank,dw1_isFormula,dw1_isSameType,dw1_isWeaklyFormulaConsistent,dw2_isWeaklyFormulaConsistent,nb1_isWeaklyFormulaConsistent,dw1_isSum
292,True,False,False,True,True,True,False,False,True,True,True,False
272,False,True,True,False,False,True,False,True,True,False,False,False
375,False,True,True,True,True,False,True,True,False,True,False,False
430,False,True,True,False,False,True,False,True,True,False,False,False
328,False,True,True,False,False,False,True,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
239,False,True,True,True,True,False,True,True,False,True,False,False
3305,False,True,True,False,False,False,True,True,False,False,False,False
317,True,False,True,True,True,True,False,True,True,True,True,False
237,False,True,True,True,False,False,True,True,True,False,True,False


In [17]:
Y_train

292     False
272     False
375     False
430     False
328     False
        ...  
239     False
3305     True
317     False
237     False
3327     True
Name: Label, Length: 231, dtype: bool

In [18]:
len(X_train), len(X_test)

(231, 58)

In [19]:
Y_train.value_counts(normalize=False)

False    176
True      55
Name: Label, dtype: int64

In [20]:
Y_test.value_counts(normalize=False)

False    45
True     13
Name: Label, dtype: int64

Naif model that evaluates all to True:

In [21]:
TP = 9
FP = 49
TN = 0
FN = 0
all = 49+9
print(f"Recall={TP/(TP+FN)}, Precision={TP/(TP+FP)}, Accuracy={(TP+TN)/all}")

Recall=1.0, Precision=0.15517241379310345, Accuracy=0.15517241379310345


## Train a LR model

Please note this is just an example, and more work is required. For example:

- AzureML AutoML could be used to try more models/parameters
- More data for training would be required
- An analysis of useful features would be welcome, as well as using more meaningful features (e.g., cell colors)

In [22]:
LRC = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr')
LRC.fit(X_train, Y_train)
LRC

LogisticRegression(multi_class='ovr', random_state=0)

In [23]:
predicted_LRC = LRC.predict(X_test)
confusion_matrix(predicted_LRC, Y_test)

array([[43,  9],
       [ 2,  4]], dtype=int64)

In [24]:
print(classification_report(Y_test, predicted_LRC))

              precision    recall  f1-score   support

       False       0.83      0.96      0.89        45
        True       0.67      0.31      0.42        13

    accuracy                           0.81        58
   macro avg       0.75      0.63      0.65        58
weighted avg       0.79      0.81      0.78        58



Save the model as a pickle file

In [25]:
with open ('./model.pkl', 'bw') as f:
    pickle.dump(LRC, f)